# DICOM to NIfTI Conversion and Other Utilities

This notebook demonstrates how to:
1. Convert a series of DICOM files to a NIfTI volume.
2. Inspect and visualize NIfTI slices.
3. Rename files according to a desired pattern.
4. Use `dcm2niix` for batch DICOM conversions.
5. Organize directories by moving/renaming files.

In [ ]:
import pydicom
import nibabel as nib
import numpy as np
import os

def dicom_to_nifti(dicom_dir, output_file):
    """
    Converts a directory of DICOM files into a single 3D NIfTI file.
    
    Args:
        dicom_dir (str): Directory containing DICOM files.
        output_file (str): Output file path for the created NIfTI.
    """
    # Load all .dcm files in the directory
    dicom_files = [pydicom.dcmread(os.path.join(dicom_dir, f)) 
                   for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    
    # Sort slices by InstanceNumber to maintain correct 3D ordering
    dicom_files.sort(key=lambda x: x.InstanceNumber)

    # Extract the pixel_array from each DICOM object and stack into a 3D volume
    pixel_arrays = [ds.pixel_array for ds in dicom_files]
    image_3d = np.stack(pixel_arrays, axis=-1)

    # Create an identity affine matrix (could be replaced with a more precise one)
    affine = np.eye(4)
    
    # Create a NIfTI image from the 3D array
    nifti_image = nib.Nifti1Image(image_3d, affine)

    # Save the NIfTI file to the specified output path
    nib.save(nifti_image, output_file)
    print(f"Saved NIfTI file to {output_file}")

# Example usage of the dicom_to_nifti function
dicom_dir = '/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977'
output_file = '/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977'
dicom_to_nifti(dicom_dir, output_file)

# Shell command using dcm2niix for DICOM to NIfTI conversion (alternative approach)
# Adjust the paths and options as necessary for your setup.
dcm2niix_command = (
    "dcm2niix -o ~'/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/"
    "PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977' "
    "~'/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/"
    "PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977'"
)
print(dcm2niix_command)  # This is just printed; remove if you'd like to run it directly in a shell.

## Visualizing a NIfTI File (Single Slice)

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

# Load a specific NIfTI file
nifti_file = '/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977/I1245977_PET_Brain_NEURACEQ_DYNAMIC_20191023141617_4.nii'
img = nib.load(nifti_file)
data = img.get_fdata()  # Convert image to a NumPy array

# If there is a 4D dataset, select the first volume
if data.ndim == 4:
    data = data[..., 0]

# Normalize the data values to [0,1] for better display
data = (data - data.min()) / (data.max() - data.min())

# Select a slice to visualize. Here, we choose one-quarter into the z-dimension.
slice_index = data.shape[2] // 4

# Display the chosen axial slice
plt.figure(figsize=(5, 5))
plt.imshow(data[:, :, slice_index], cmap='gray')
plt.title("Axial View (Slice Index = 1/4 of total)")
plt.axis('off')
plt.show()

## Inspect the Shape of the NIfTI Data

In [ ]:
import nibabel as nib

# Load the same (or different) NIfTI file
nifti_file = '/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977/I1245977_PET_Brain_NEURACEQ_DYNAMIC_20191023141617_4.nii'
img = nib.load(nifti_file)
data = img.get_fdata()

# Print the shape of the data to understand dimensions (e.g., X, Y, Z)
print("Shape of the data:", data.shape)

# If data is 3D, the number of slices is the size along the z-axis
num_slices = data.shape[2]
print("Number of slices:", num_slices)

## Plotting Multiple NIfTI Slices in a Grid

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

# Load the NIfTI file
nifti_file = '/home/hamse/Documents/dataset/AD ADNI3 FBB/ADNI/013_S_6768/PET_Brain_NEURACEQ_DYNAMIC_FBB/2019-10-23_14_38_38.0/I1245977/I1245977_PET_Brain_NEURACEQ_DYNAMIC_20191023141617_4.nii'
img = nib.load(nifti_file)
data = img.get_fdata()

# Normalize for visualization
data = (data - data.min()) / (data.max() - data.min())

# Determine how many slices there are
num_slices = data.shape[2]
cols = 10  # Number of columns in the display grid
rows = int(np.ceil(num_slices / cols))  # Number of rows needed

# Create a figure with a grid of subplots
fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
axes = axes.flatten()  # Flatten to make it easy to iterate

# Plot each slice in a subplot
for i in range(num_slices):
    axes[i].imshow(data[:, :, i], cmap='gray')
    axes[i].axis('off')  # Hide axis ticks for a clean look

# Hide any empty plots if num_slices < rows * cols
for i in range(num_slices, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Using `dcm2niix` in Python (Subprocess for Batch Conversion)

In [ ]:
import os
import subprocess

# Define the input and output directories
dcim_dir = "/home/hamse/Documents/dataset 2.0/N/dcim"
niftis_dir = "/home/hamse/Documents/dataset 2.0/N/nftis"

# Iterate over folders in the DICOM directory
for folder in os.listdir(dcim_dir):
    folder_path = os.path.join(dcim_dir, folder)
    
    # Only process if it's a valid directory
    if os.path.isdir(folder_path):
        # Input DICOM folder path
        inp = folder_path
        
        # Corresponding output folder path for NIfTI files
        out = os.path.join(niftis_dir, folder)
        
        # Create the output directory if it does not exist
        os.makedirs(out, exist_ok=True)
        
        # Construct and run the dcm2niix command
        command = f'dcm2niix -o "{out}" "{inp}"'
        subprocess.run(command, shell=True)

## Checking for Duplicate Items in a List

In [ ]:
a = ['I1262409', 'I1370312', 'I966135', 'I1053175', 'I1281017', 'I1346704',
     'I1245977', 'I1019088', 'I1369689', 'I1040762', 'I1476921', 'I1559915',
     'I1265211', 'I1358480', 'I1352931', 'I1326076', 'I1268721', 'I1617994',
     'I1181917', 'I1049841', 'I1141914', 'I1326077', 'I1268722', 'I994961',
     'I1022107', 'I1528084', 'I1169971', 'I1584484', 'I1158613', 'D10245957',
     'I1156484', 'I1083504', 'I1146177', 'I1222086', 'I903164', 'I1300334',
     'I1158155', 'I1049430', 'I1278129', 'I1187871', 'I1610089', 'I1585187',
     'I1166395', 'I1674817', 'I1185216', 'I1434190', 'I1494474', 'I1139340',
     'I1425386', 'I1489941', 'I1229514', 'I1126661', 'I1178513', 'I1489940',
     'I1227193', 'I858116', 'I1484506', 'I1044453', 'I1061155', 'I1285568']

import collections
# Print list of items that appear more than once
duplicates = [item for item, count in collections.Counter(a).items() if count > 1]
print(duplicates)

## Listing Folders in a Directory

In [ ]:
import os

directory = "/home/hamse/Documents/dataset 2.0/AD"
folder_names = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
print(folder_names)

## Moving Files from Subdirectories to a Main Directory

In [ ]:
import os
import shutil

main_dir = "/home/hamse/Documents/dataset 2.0/N/nftis"

# Walk through the entire directory tree
for root, dirs, files in os.walk(main_dir):
    for file in files:
        file_path = os.path.join(root, file)
        # Move the file to the main directory (main_dir)
        shutil.move(file_path, os.path.join(main_dir, file))

# Now remove the empty subdirectories
for dir in os.listdir(main_dir):
    dir_path = os.path.join(main_dir, dir)
    if os.path.isdir(dir_path):
        os.rmdir(dir_path)

## Renaming JSON Files by Extracting the First Part Before an Underscore

In [ ]:
import os

json_dir = "/home/hamse/Documents/dataset 2.0/N/json"

for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        # Extract new name from the first part before the underscore
        new_name = filename.split('_')[0] + '.json'
        
        original_path = os.path.join(json_dir, filename)
        new_path = os.path.join(json_dir, new_name)
        
        os.rename(original_path, new_path)

print("JSON files renamed successfully.")

## Listing Folders in Another Directory (MCI)

In [ ]:
import os

directory = "/home/hamse/Documents/dataset 2.0/MCI"
folder_names = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
print(folder_names)

## Renaming `.nii` Files by Keeping Only the Part Before the Underscore

In [ ]:
import os

niftis_dir = "/home/hamse/Documents/dataset 2.0/N/nftis"

for filename in os.listdir(niftis_dir):
    if filename.endswith('.nii'):
        new_name = filename.split('_')[0] + '.nii'
        
        original_path = os.path.join(niftis_dir, filename)
        new_path = os.path.join(niftis_dir, new_name)
        
        os.rename(original_path, new_path)

print("Files renamed successfully.")

*(Empty cell for future usage or notes.)*